In [1]:
# resnet50, inceptionv3, xception, vgg19, densenet201 
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import pandas as pd
import os
import re

models = []
models.append(ResNet50(weights='imagenet'))
models.append(InceptionV3(weights='imagenet'))

C:\Users\osk\Anaconda3\envs\tensorflow-gpu_20180508\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [9]:
targetdir = './ants/'
df = pd.DataFrame()
df['image'] = os.listdir(targetdir)
df['image'] = df['image'].apply(lambda x: targetdir+str(x))

df.head()

,image
0,./ants/10308379_1b6c72e180.jpg
1,./ants/11381045_b352a47d8c.jpg
2,./ants/119785936_dd428e40c3.jpg
3,./ants/147542264_79506478c2.jpg
4,./ants/17081114_79b9a27724.jpg


In [10]:
def predict_imagenet(img_path, models, topk=10):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = []
    for model in models:
        pred = model.predict(x)
        preds.append(decode_predictions(pred, top=topk)[0])
    return preds

# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
#print('Predicted:', decode_predictions(preds, top=3)[0])

df['preds'] = df['image'].apply(lambda x: predict_imagenet(x, models))
df.head()

,image,preds
0,./ants/10308379_1b6c72e180.jpg,"[[(n02219486, ant, 0.6135076), (n03388043, fou..."
1,./ants/11381045_b352a47d8c.jpg,"[[(n12620546, hip, 0.59364426), (n07768694, po..."
2,./ants/119785936_dd428e40c3.jpg,"[[(n02219486, ant, 0.25583923), (n02206856, be..."
3,./ants/147542264_79506478c2.jpg,"[[(n02219486, ant, 0.9924057), (n12620546, hip..."
4,./ants/17081114_79b9a27724.jpg,"[[(n02219486, ant, 0.24959956), (n03000134, ch..."


In [16]:
temp = df['preds'][0]
temp

[[('n02219486', 'ant', 0.6135076),
  ('n03388043', 'fountain', 0.2282419),
  ('n04589890', 'window_screen', 0.043679826),
  ('n03530642', 'honeycomb', 0.040876117),
  ('n04153751', 'screw', 0.020330062),
  ('n03000134', 'chainlink_fence', 0.006978868),
  ('n04040759', 'radiator', 0.0055563976),
  ('n02727426', 'apiary', 0.0054270625),
  ('n03804744', 'nail', 0.0045063407),
  ('n02264363', 'lacewing', 0.0034495932)],
 [('n03000134', 'chainlink_fence', 0.98579454),
  ('n03950228', 'pitcher', 0.014176),
  ('n06359193', 'web_site', 2.9432224e-05),
  ('n04131690', 'saltshaker', 3.8369335e-10),
  ('n04286575', 'spotlight', 5.9625936e-14),
  ('n04404412', 'television', 3.0905454e-14),
  ('n02403003', 'ox', 2.0669312e-14),
  ('n03047690', 'clog', 7.791025e-15),
  ('n02114367', 'timber_wolf', 2.002092e-15),
  ('n02841315', 'binoculars', 1.5946907e-16)]]

In [15]:
for t in range(5):
    for i in range(len(models)):
        df['imagenet_preds_top'+str(t+1)+'_'+str(i)] = [df['preds'][j][i][t][2] for j in range(df.shape[0])]

df.head()

,image,preds,imagenet_preds_top1_0,imagenet_preds_top1_1,imagenet_preds_top2_0,imagenet_preds_top2_1,imagenet_preds_top3_0,imagenet_preds_top3_1,imagenet_preds_top4_0,imagenet_preds_top4_1,imagenet_preds_top5_0,imagenet_preds_top5_1
0,./ants/10308379_1b6c72e180.jpg,"[[(n02219486, ant, 0.6135076), (n03388043, fou...",0.613508,0.985795,0.228242,1.417600e-02,0.043680,2.943222e-05,0.040876,3.836934e-10,0.020330,5.962594e-14
1,./ants/11381045_b352a47d8c.jpg,"[[(n12620546, hip, 0.59364426), (n07768694, po...",0.593644,0.830617,0.176894,1.693833e-01,0.109483,3.201873e-08,0.035305,1.326941e-08,0.034756,4.713917e-09
2,./ants/119785936_dd428e40c3.jpg,"[[(n02219486, ant, 0.25583923), (n02206856, be...",0.255839,0.814238,0.178143,1.857271e-01,0.116703,3.489425e-05,0.093549,5.863076e-10,0.058584,6.118975e-11
3,./ants/147542264_79506478c2.jpg,"[[(n02219486, ant, 0.9924057), (n12620546, hip...",0.992406,0.999999,0.002162,6.003089e-07,0.001593,9.705174e-14,0.001273,7.085916e-17,0.001108,1.697638e-17
4,./ants/17081114_79b9a27724.jpg,"[[(n02219486, ant, 0.24959956), (n03000134, ch...",0.249600,0.687288,0.199474,1.683310e-01,0.084262,1.112581e-01,0.054740,1.548801e-02,0.022768,7.383630e-03
